# Zuber -- 2017 Chicago Taxi Trip Analysis

## Introduction

Below we will be looking at three datasets involving taxi company information gathered for Chicago in November 2017. The first two datasets will require some exploratory data analysis, while the third has already been specifically filtered to test the hypothesis that the average duration of rides from the Loop to O'Hare International Airport changes on rainy Saturdays. The bulk of the below analysis will be on cleaning and understanding the first two datasets, but we will conclude with our test of the above hypothesis on the third dataset.

## Initializing

In [1]:
# Loading all the libraries
import pandas as pd
from scipy import stats as st

# Graphing libraries
import plotly.express as px

In [2]:
# Load Dataframe on company information
#     company_name           :    taxi company name
#     trips_amount           :    the number of rides for each taxi company on Nov 15-16, 2017
company_df = pd.read_csv('./datasets/project_sql_result_01.csv') 

# Load Dataframe on neighborhood information
#     dropoff_location_name  :    neighborhoods where rides ended
#     average_trips          :    the average number of rides that ended in each neighborhood
neigh_df = pd.read_csv('./datasets/project_sql_result_04.csv')

# Load Dataframe on information to answer rainy hypothesis
#     start_ts               :    pickup date and time
#     weather_conditions     :    weather conditions at ride start ('Good' or 'Bad')
#     duration_seconds       :    ride duration in seconds
rain_df = pd.read_csv('./datasets/project_sql_result_07.csv')

### The `company_df` Dataframe

In [3]:
# Quick view
company_df.info()
display(company_df.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


,company_name,trips_amount
8,Star North Management LLC,7455
52,3721 - Santamaria Express,14
36,3591 - 63480 Chuks Cab,37
35,6742 - 83735 Tasha Ride Inc,39
55,6057 - 24657 Richard Addo,13


In [4]:
# Check for duplicates and remove if present
if len(company_df[company_df.duplicated()]) > 0:
    print('Detected', len(company_df[company_df.duplicated()]), 'duplicates. Removing all.')
    print('')
    company_df = company_df[~company_df.duplicated()]
else:
    print('No duplicate rows detected.')

No duplicate rows detected.


The dataframe looks clean, with all correct data types and no missing values or duplicates.

### The `neigh_df` Dataframe

In [5]:
# Quick view
neigh_df.info()
display(neigh_df.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dropoff_location_name  94 non-null     object 
 1   average_trips          94 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.6+ KB


,dropoff_location_name,average_trips
76,Washington Heights,9.133333
59,Chatham,23.200000
79,Fuller Park,8.166667
84,West Pullman,6.466667
26,Hyde Park,255.133333


In [6]:
# Check for duplicates and remove if present
if len(neigh_df[neigh_df.duplicated()]) > 0:
    print('Detected', len(neigh_df[neigh_df.duplicated()]), 'duplicates. Removing all.')
    print('')
    neigh_df = neigh_df[~neigh_df.duplicated()]
else:
    print('No duplicate rows detected.')

No duplicate rows detected.


The dataframe looks clean, with all correct data types and no missing values or duplicates.

### The `rain_df` Dataframe

In [7]:
# Quick view
rain_df.info()
display(rain_df.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   start_ts            1068 non-null   object 
 1   weather_conditions  1068 non-null   object 
 2   duration_seconds    1068 non-null   float64
dtypes: float64(1), object(2)
memory usage: 25.2+ KB


,start_ts,weather_conditions,duration_seconds
345,2017-11-11 17:00:00,Good,2271.0
699,2017-11-18 12:00:00,Bad,2640.0
718,2017-11-04 20:00:00,Good,2220.0
555,2017-11-11 07:00:00,Good,1203.0
418,2017-11-04 07:00:00,Good,1563.0


In [8]:
# We need to convert to datetime for the 'start_ts' column, and to int64 for the 'duration_seconds' column
rain_df['start_ts'] = pd.to_datetime(rain_df['start_ts'])
rain_df['duration_seconds'] = rain_df['duration_seconds'].astype('int64')

# Check for duplicates and remove if present
if len(rain_df[rain_df.duplicated()]) > 0:
    print('Detected', len(rain_df[rain_df.duplicated()]), 'duplicates. Removing all.')
    print('')
    rain_df = rain_df[~rain_df.duplicated()]
else:
    print('No duplicate rows detected.')

# Display changes
rain_df.info()

Detected 197 duplicates. Removing all.

<class 'pandas.core.frame.DataFrame'>
Index: 871 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   start_ts            871 non-null    datetime64[ns]
 1   weather_conditions  871 non-null    object        
 2   duration_seconds    871 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 27.2+ KB


## EDA for the `company_df` and `neigh_df` Dataframes

### Top Companies

In [9]:
# Show top 10 Companies
print('')
print('Top 10 Companies by Number of Trips for Nov 15-16:')
display(company_df.sort_values(by=['trips_amount'], ascending=False).head(10))


Top 10 Companies by Number of Trips for Nov 15-16:


,company_name,trips_amount
0,Flash Cab,19558
1,Taxi Affiliation Services,11422
2,Medallion Leasing,10367
3,Yellow Cab,9888
4,Taxi Affiliation Service Yellow,9299
5,Chicago Carriage Cab Corp,9181
6,City Service,8448
7,Sun Taxi,7701
8,Star North Management LLC,7455
9,Blue Ribbon Taxi Association Inc.,5953


The top three companies gave over 10,000 trips each in Chicago. The top two companies have 'Cab' or 'Taxi' in their name. It seems there is an advantage to advertising a company's purpose in its title. Let's look at this visually:

In [10]:
# First plot is a bar graph showing top 10 neighborhoods
fig1 = px.bar(company_df.sort_values(by=['trips_amount'], ascending=False).head(10), x='company_name', y='trips_amount', 
            labels={'company_name': ' Company Name', 'trips_amount': 'Total Trips for Nov 15-16'},
            title='Top 10 Chicago Taxi Companies for Nov 15-16')
fig1.show()

# Split performance data by whether company has 'Taxi' or 'Cab' in its name
company_df['taxi_cab_in_name'] = company_df['company_name'].str.contains('Taxi|Cab')
fig = px.box(company_df, x='taxi_cab_in_name', y='trips_amount', 
             labels={'taxi_cab_in_name': 'Whether "Taxi" or "Cab" is in the Name', 'trips_amount': 'Total Trips for Nov 15-16'},
             points='all', title='Nov 15-16 Performance by Presence of \'Taxi\' or \'Cab\' in Company Name')
fig.show()

There certainly appears to be an advantage simply by having Taxi or Cab in the company name. The three companies that did do well without Taxi or Cab in their name are outliers in their group.

### Top Neighborhoods

In [11]:
# Show top 10 Destination Neighborhoods
print('')
print('Top 10 Destination Neighborhoods by Number of Trips for Nov 15-16:')
display(neigh_df.sort_values(by=['average_trips'], ascending=False).head(10))


Top 10 Destination Neighborhoods by Number of Trips for Nov 15-16:


,dropoff_location_name,average_trips
0,Loop,10727.466667
1,River North,9523.666667
2,Streeterville,6664.666667
3,West Loop,5163.666667
4,O'Hare,2546.900000
5,Lake View,2420.966667
6,Grant Park,2068.533333
7,Museum Campus,1510.000000
8,Gold Coast,1364.233333
9,Sheffield & DePaul,1259.766667


Destination neighborhoods are dominated by a number of large neighborhoods with thousands of daily trips, while over 65 neighborhoods have less than 250 daily trips. We can see visually first a list of all large neighborhoods (top 10) and then a distribution of all neighborhoods:

In [12]:
# First plot is a bar graph showing top 10 neighborhoods
fig1 = px.bar(neigh_df.query('average_trips > 1259.765'), x='dropoff_location_name', y='average_trips', 
            labels={'dropoff_location_name': ' Neighborhood', 'average_trips': 'Average Daily Trips in November'},
            title='Top 10 Destination Neighborhoods by Average Daily Trips')
fig1.show()

# Second plot is a histogram showing overall distribution across all neighborhoods
fig = px.histogram(neigh_df, x='average_trips', nbins = 30, 
            labels={'average_trips': 'Average Daily Trips in November'},
            title='Distribution of Trips Across All Neighborhoods as Destinations')
fig.show()

We should recall that this data is filtering trips by destination location, not by origin location. Neighborhoods are more likely to be destinations when they are high density areas of work, leisure, or especially events, and indeed the top four destination neighborhoods correspond to the areas in and immediately around downtown Chicago.

## Testing A Hypothesis on Rain

We have a hypothesis to test: we think that the average duration of rides from the Loop to O'Hare International Airport changes on rainy Saturdays. Our null hypothesis in this instance would be quite the opposite: that rain has no effect on the average duration of rides from the Loop to O'Hare International Airport.

The trip from the Loop to O'Hare is commonly made, as it represents traveling from downtown Chicago to one of the busiest airports in the U.S. Rain could feasibly raise or lower the average duration of rides, as while some people might not ride as much or take shorter rides, those that do ride would have to fight slower traffic and worse conditions. It could be argued logically either way.

Our third dataframe, named `rain_df`, has already been filtered to include only trips made from the Loop to O'Hare on Saturdays, so we are ready to perform the test. All we must do is split the dataframe by the 'weather_conditions' column, which will indicate 'Bad' if there was rain or a storm and 'Good' if not. We are therefore performing a two-sided t-test of two population means. We don't have a reason right now to consider that the rain increases or lowers the number of trips, so we are merely testing for the presence of a statistically-significant effect. 

In [13]:
# Significance level set to high
alpha = 0.01 

# Two-sided t-test of two population means. We can leave equal_var True as these are pulled from the same set of trip data.
results = st.ttest_ind(
    rain_df.query('weather_conditions == "Good"')['duration_seconds'],
    rain_df.query('weather_conditions == "Bad"')['duration_seconds'],
    equal_var=True
)

good_mean = rain_df.query('weather_conditions == "Good"')['duration_seconds'].mean()
bad_mean = rain_df.query('weather_conditions == "Bad"')['duration_seconds'].mean()

# Return
print('Good Weather Condition Average Duration in Seconds:', good_mean)
print('Bad Weather Condition Average Duration in Seconds:', bad_mean)
print('')
print('p-value: ', results.pvalue)
if results.pvalue < alpha:
    print("We reject the null hypothesis.")
else:
    print("We can't reject the null hypothesis.")

Good Weather Condition Average Duration in Seconds: 2032.253112033195
Bad Weather Condition Average Duration in Seconds: 2409.2297297297296

p-value:  7.397770692813658e-08
We reject the null hypothesis.


It seems there is a rather significant difference when the element of rain is introduced. Comparing the population means here indicates that rain slows the average duration. Traffic cannot he when roads are slick, and some roads become closed or diverted as a result of heavy rainfall.

## Conclusion



Chicago cab trips during Nov 2017 were dominated by a handful of companies, with those companies having "Taxi" or "Cab" in their name possessing a competitive advantage. There were still dozens of smaller companies with hundreds or even thousands of rides during that month vying for the remaining market share.

Nearly 77 formally-defined neighborhoods exist in Chicago and those that are closer to downtown naturally see more traffic. The vast majority of neighborhoods see fewer than 300 trips in a month.

Finally, weather will naturally affect consumer behavior. On the metric of average trip length, we learned that rain will lengthen cab trips.